# Decorators in Python

## Functions are objects too

In Python, functions are "first class objects". That is, they can be:
- created
- destroyed
- passed to a function
- returned as a value
- assigned variable names

In [3]:
# Created and destroyed
def func():
    print('func')

func()
del func
func()

func


NameError: name 'func' is not defined

In [13]:
# Passed to a function
def wrapper(x):
    x()
    some_func()

def new_func():
    wrapper(some_func)

def some_func():
    print('func')

new_func()
# wrapper(some_func)

func
func


In [21]:
def dumb(x):
    return x

x = 1
x = dumb(1)

In [26]:
# Returned as a value and assigned to variable names
def wrapper(x):
    return x

def func():
    print('func')

f = wrapper(func)
print(f)
f()

<function func at 0x00000170E05F0820>
func


In [28]:
# Can be nested in other functions
def decorator(x):
    def wrapper():
        print('In wrapper')
        x()
    return wrapper

def func():
    print('func')

d = decorator(func)    # d will be associated with wrapper
d()
func = decorator(func)
func()

In wrapper
func


## Function Decorators

### Why
Allows a user to add new functionality to an existing function without modifying it (e.g. timing, logging)

### What

A decorator is a function that 
- takes a function as an argument (e.g. func)
- wraps the call to func inside another function (e.g. wrapper)
- returns wrapper

### How
By assigning func to a call to decorator, func is replaced with the wrapper
```
func = decorator(func)
```
Calling the decorated func will execute statements in func and in wrapper.
This means wrapper functionality is added to any function passed to the decorator.

In [30]:
def decorator(func):
    def wrapper():
        print(f'Wrapping {func.__name__} ')
        func()
    return wrapper

def print_this():
    print('This')

print(f'Before: {print_this}')      # Show the name of the print_this function before
                                    # the call to the wrapper
print_this = decorator(print_this)  # Replace print_this with wrapper returned by decorator

print(f'After: {print_this}')       # Show the name of the print_this function after
                                    # the call to the wrapper
print_this()

Before: <function print_this at 0x00000170E13FB310>
After: <function decorator.<locals>.wrapper at 0x00000170E148A3A0>
Wrapping print_this 
This


### @ symbol
Python has a special syntax to replace the statement

func = decorator(func)

Add @<decorator_name> above any function definition (e.g. @decorator)

In [31]:
@decorator
def print_this():
    print('This')

@decorator
def print_that():
    print('That')

print_this()
print_that()

Wrapping print_this 
This
Wrapping print_that 
That


In [33]:
import time

def time_it(func):
    def inner():
        start = time.time()
        func()
        print(f'{func.__name__}:  {time.time() - start} sec')
    return inner

def demo():
    time.sleep(2)

def demo2():
    time.sleep(1)

demo = time_it(demo)
demo()

demo:  2.005791187286377 sec


In [34]:
@time_it
def demo():
    time.sleep(2)

demo()

demo:  2.0135467052459717 sec


### Decorator with arguments

The call to the decorated function is to the wrapper function so:
- wrapper will have the parameters matching the wrapped function
- the arguments will be passed to the wrapped function

In [35]:
def show_args(func):
    def wrapper(a, b):
        print(f'args: {a}, {b}')
        func(a, b)
    return wrapper
        
def demo(a, b):
    print(f'a = {a}, b = {b}')

demo = show_args(demo)   # Replace demo function with wrapper function
                         # returned by show_args decorator
demo(1, 2)               # Call to decorated function

args: 1, 2
a = 1, b = 2


In [ ]:
@show_args
def demo(a, b):
    print(f'a = {a}, b = {b}')

demo(1, 2)

### Decorators for functions with variable number of arguments

- *args parameter is used for variable number of postional arguments
- **kwargs parameeter is used for variable number of keyword arguments
- As with all Python function calls, positional must precede keyword arguments

In [ ]:
def show_args(func):
    def wrapper(*args, **kwargs):
        func(*args, **kwargs)
        args = [a for a in args]
        kwargs = {k:v for k, v in kwargs.items()}
        print(f'*args: {args}; **kwargs: {kwargs}')
        
    return wrapper
        
@show_args
def no_args():
    print('no_args')

no_args()

@show_args
def positional(a, b):
    print('positional')

positional(1, 2)

@show_args
def keyword(a, b):
    print('keyword')

keyword(b=1, a=2)

@show_args
def postional_and_keyword(a, b, c, d):
    print('postional_and_keyword')

postional_and_keyword(1, 2, d=4, c=3)

### Decorating functions with return values

The call to the decorated function is to the wrapper function so wrapper function will return the value returned by the wrapped function
  

In [ ]:
def return_args(func):
    def wrapper(a, b):
        print(f'args: {a}, {b}')
        return func(a, b)
    return wrapper
        
def demo(a, b):
    return f'a = {a}, b = {b}'

demo = return_args(demo)    # Replace demo function with wrapper function
                            # returned by return_args decorator
print(demo(1, 2))           # Call to decorated function

### Using wraps decorator from functools to get name of wrapped function

Provides the name of the wrapped function rather than the wrapper in a decorator

In [ ]:
def decorator(func):
    def wrapper():
        print(f'Wrapping {func.__name__} ')
        func()
    return wrapper

def print_this():
    print('This')

print(f'Before: {print_this}')      # Show the name of the print_this function before
                                    # the call to the wrapper
print_this = decorator(print_this)  # Replace print_this with wrapper returned by decorator

print(f'After: {print_this}')       # Show the name of the print_this function after
                                    # the call to the wrapper
print_this()

In [ ]:
from functools import wraps
def decorator(func):
    @wraps(func)
    def wrapper():
        print(f'Wrapping {func.__name__} ')
        func()
    return wrapper

def print_this():
    print('This')

print(f'Before: {print_this}')      # Show the name of the print_this function before
                                    # the call to the wrapper
print_this = decorator(print_this)  # Replace print_this with wrapper returned by decorator

print(f'After: {print_this}')       # Show the name of the print_this function after
                                    # the call to the wrapper
print_this()